### analysis pipeline

In [11]:
!duckdb scalable.db ".tables"
!duckdb scalable.db "SELECT COUNT(*) FROM pg_proc;"

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     2709     │
└──────────────┘


In [12]:
import duckdb
import os
import shutil
import signal
import sys
import hashlib

con = None

def get_file_hash(filepath):
    hasher = hashlib.sha256()
    with open(filepath, 'rb') as f:
        while chunk := f.read(65536):
            hasher.update(chunk)
    return hasher.hexdigest()

def signal_handler(sig, frame):
    global con
    if con: con.close()
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

def run_pipeline(input_dir, db_path="../data/outputs/scalable.db", bronze_dir="../data/outputs/bronze_listens", gold_dir="data/outputs/gold_exports", reset=False):
    global con
    if reset:
        for d in [bronze_dir, gold_dir]:
            if os.path.exists(d): shutil.rmtree(d)
        if os.path.exists(db_path): os.remove(db_path)
    
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    os.makedirs(bronze_dir, exist_ok=True)
    os.makedirs(gold_dir, exist_ok=True)
    con = duckdb.connect(db_path)

    try:
        con.execute("SET memory_limit = '4GB'; SET threads = 4; SET preserve_insertion_order = false;")

        con.execute("CREATE TABLE IF NOT EXISTS processed_files (content_hash VARCHAR PRIMARY KEY, filename VARCHAR, processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)")
        processed_hashes = set(row[0] for row in con.execute("SELECT content_hash FROM processed_files").fetchall())
        
        all_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.txt')]
        for file_name in all_files:
            full_path = os.path.join(input_dir, file_name)
            file_hash = get_file_hash(full_path)
            if file_hash in processed_hashes: continue 

            con.execute(f"""
                COPY (SELECT *, (track_metadata->>'track_name') as track_name, (track_metadata->>'artist_name') as artist_name
                FROM read_json('{full_path}', format='newline_delimited',
                columns={{'user_name': 'VARCHAR', 'listened_at': 'BIGINT', 'recording_msid': 'VARCHAR', 'track_metadata': 'JSON'}}))
                TO '{bronze_dir}' (FORMAT 'PARQUET', PARTITION_BY (user_name), OVERWRITE_OR_IGNORE 1)
            """)
            con.execute("INSERT INTO processed_files (content_hash, filename) VALUES (?, ?)", [file_hash, file_name])

        con.execute("""
            CREATE TABLE IF NOT EXISTS silver_listens (
                user_name VARCHAR, listened_at BIGINT, recording_msid VARCHAR,
                artist_name VARCHAR, track_name VARCHAR, listened_date DATE,
                UNIQUE(user_name, listened_at)
            )
        """)
        
        if any(os.scandir(bronze_dir)):
            con.execute(f"""
                INSERT INTO silver_listens (user_name, listened_at, recording_msid, artist_name, track_name, listened_date)
                SELECT user_name, listened_at, recording_msid, artist_name, track_name, to_timestamp(listened_at)::DATE
                FROM read_parquet('{bronze_dir}/**/*.parquet') ON CONFLICT (user_name, listened_at) DO NOTHING
            """)

        con.execute("CREATE OR REPLACE VIEW gold_top_10 AS SELECT user_name, COUNT(*) as cnt FROM silver_listens GROUP BY 1 ORDER BY 2 DESC LIMIT 10")
        con.execute("""
            CREATE OR REPLACE VIEW gold_user_top_days AS
            WITH daily AS (SELECT user_name, listened_date, COUNT(*) as cnt FROM silver_listens GROUP BY 1, 2)
            SELECT user_name as user, cnt as number_of_listens, listened_date as date
            FROM (SELECT *, ROW_NUMBER() OVER(PARTITION BY user_name ORDER BY cnt DESC) as r FROM daily)
            WHERE r <= 3 ORDER BY user ASC, number_of_listens DESC
        """)
        con.execute("""
            CREATE OR REPLACE VIEW gold_active_user_trends AS
            WITH daily_u AS (SELECT listened_date as d, user_name as u FROM silver_listens GROUP BY 1, 2),
                 all_d AS (SELECT DISTINCT listened_date as d FROM silver_listens),
                 total AS (SELECT COUNT(DISTINCT user_name) as total_cnt FROM silver_listens)
            SELECT ad.d as date, COUNT(DISTINCT du.u) as number_active_users,
                   ROUND(COUNT(DISTINCT du.u) * 100.0 / (SELECT total_cnt FROM total), 2) as percentage_active_users
            FROM all_d ad JOIN daily_u du ON du.d BETWEEN ad.d - INTERVAL 6 DAY AND ad.d GROUP BY 1 ORDER BY 1
        """)

        con.execute(f"COPY gold_user_top_days TO '{gold_dir}/user_peaks.parquet' (FORMAT 'PARQUET')")
        con.execute(f"COPY gold_active_user_trends TO '{gold_dir}/active_user_trends.parquet' (FORMAT 'PARQUET')")

        b_cnt = con.execute(f"SELECT COUNT(*) FROM read_parquet('{bronze_dir}/**/*.parquet')").fetchone()[0]
        s_cnt = con.execute("SELECT COUNT(*) FROM silver_listens").fetchone()[0]
        g_cnt = con.execute("SELECT COUNT(*) FROM gold_user_top_days").fetchone()[0]
        
        print(f"\nAudit:\n- Bronze: {b_cnt}\n- Silver: {s_cnt}\n- Gold:   {g_cnt}")
        print(f"\nBI files ready in: {gold_dir}/")

    finally:
        if con: con.close()

if __name__ == "__main__":
    run_pipeline("../data/inputs", reset=False)


Audit:
- Bronze: 4
- Silver: 4
- Gold:   2

BI files ready in: data/outputs/gold_exports/


In [14]:
if __name__ == "__main__":
    # Ensure paths are consistent
    INPUT_PATH = "../data/inputs"
    DB_PATH = "../data/outputs/scalable.db"

    # 1. Run pipeline (Bronze -> Silver -> Gold)
    run_pipeline(INPUT_PATH, db_path=DB_PATH, reset=True)
    
    print("\n" + "=" * 60)
    print("TASK #2 QUERIES - Using duckdb.query()")
    print("=" * 60)
    
    # 2. Connect to the actual file created by run_pipeline
    db = duckdb.connect(DB_PATH, read_only=True)
    
    # A1: Top 10 users
    print("\nA1. Top 10 users by songs listened to:")
    # Use duckdb.query(sql, connection=db) as requested
    duckdb.query("SELECT user_name, COUNT(*) AS listen_count FROM silver_listens GROUP BY 1 ORDER BY 2 DESC LIMIT 10", connection=db).show()
    
    # A2: Users on March 1st, 2019
    print("\nA2. Users on 2019-03-01:")
    duckdb.query("SELECT COUNT(DISTINCT user_name) as user_count FROM silver_listens WHERE listened_date = '2019-03-01'", connection=db).show()
    
    # A3: First song per user
    print("\nA3. First song listened to by each user:")
    duckdb.query("""
        SELECT user_name, track_name, artist_name, listened_date
        FROM (SELECT *, ROW_NUMBER() OVER(PARTITION BY user_name ORDER BY listened_at ASC) as rn FROM silver_listens)
        WHERE rn = 1 ORDER BY user_name
    """, connection=db).show()

    # A4: Daily Active Users (7-day rolling window)
    print("\nA4. Daily Active Users (7-day window):")
    # Fixed syntax: '6 DAY' instead of '6 DAYS'
    duckdb.query("""
        WITH daily_users AS (
            SELECT listened_date, user_name FROM silver_listens GROUP BY 1, 2
        ),
        all_dates AS (
            SELECT DISTINCT listened_date FROM silver_listens
        ),
        active_counts AS (
            SELECT 
                curr.listened_date,
                COUNT(DISTINCT past.user_name) as number_active_users
            FROM all_dates curr
            LEFT JOIN daily_users past 
            ON past.listened_date BETWEEN (curr.listened_date - INTERVAL 6 DAY) AND curr.listened_date
            GROUP BY curr.listened_date
        ),
        total_users AS (
            SELECT COUNT(DISTINCT user_name) as total FROM silver_listens
        )
        SELECT 
            listened_date as date,
            number_active_users,
            round((number_active_users::FLOAT / (SELECT total FROM total_users)) * 100, 2) as percentage_active_users
        FROM active_counts
        ORDER BY date ASC
    """, connection=db).show()
    
    db.close()


Audit:
- Bronze: 4
- Silver: 4
- Gold:   2

BI files ready in: data/outputs/gold_exports/

TASK #2 QUERIES - Using duckdb.query()

A1. Top 10 users by songs listened to:
┌───────────┬──────────────┐
│ user_name │ listen_count │
│  varchar  │    int64     │
├───────────┼──────────────┤
│ NichoBI   │            4 │
└───────────┴──────────────┘


A2. Users on 2019-03-01:
┌────────────┐
│ user_count │
│   int64    │
├────────────┤
│          0 │
└────────────┘


A3. First song listened to by each user:
┌───────────┬──────────────────────────┬─────────────┬───────────────┐
│ user_name │        track_name        │ artist_name │ listened_date │
│  varchar  │         varchar          │   varchar   │     date      │
├───────────┼──────────────────────────┼─────────────┼───────────────┤
│ NichoBI   │ Boots of Spanish Leather │ Bob Dylan   │ 2019-04-14    │
└───────────┴──────────────────────────┴─────────────┴───────────────┘


A4. Daily Active Users (7-day window):
┌────────────┬──────────────